Import our dependencies and the Minitorch from my github

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import requests
import sys
import os
from tqdm import tqdm


from engine.minitorch import Tensor,optim

I will put a seed right now for the reproducibility of the project.

In [2]:
np.random.seed(3329)

This are the variables that we need in order to create the data folder with the dataset inside,because I can't upload the repository with the dataset of Shakespeare pre-charged.

In [3]:
data_directory = "data"
file_path = os.path.join(data_directory, "input.txt")
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"

This will create a data folder if it doesn't exists

In [4]:
if not os.path.exists(data_directory):
    
    os.makedirs(data_directory)

if the inside of the data folder that we just created is empty we throw a request to the url for the dataset of Shakespeare of the github of Karpathy,raise for status to inform us what happened with the request and then open it indicating to the computer that need to transform it binary to avoid missinterpretations of characters and then finally write it down to the "input.txt" inside our data folder

In [5]:
if not os.path.exists(file_path):
    
    response = requests.get(url)
    
    response.raise_for_status()


    with open(file_path, "wb") as f:

        f.write(response.content)

        